In [1]:
import os
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import datetime as dt

# set option
pd.set_option('display.max_rows', 999)
pd.set_option('display.max_columns', 999)

# Get data

In [2]:
url = 'https://www.worldometers.info/coronavirus'

In [3]:
# get data from url
req = requests.get(url)
soup = BeautifulSoup(req.content, 'html.parser')
content = soup.find('table', id = 'main_table_countries_yesterday2')

# put data into DataFrame
data = []
attributes = [item.text.replace('\xa0', ' ').replace('\n', '') for item in content.find_all('th')]
for i in content.find_all('tr')[1:]:
    data.append([j.text for j in i.find_all('td')])
df = pd.DataFrame(data, columns = attributes)
df = df[df['#'] != ''].set_index('#')
df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
#,,,,,,,,,,,,,,,,,,,,,
1,China,"203,334","+2,680","4,776",+51,"169,380","+2,982","29,178",274,141,3,"160,000,000","111,163","1,439,323,776",Asia,"7,079","301,366",9,2,0.04,20
2,USA,"82,747,175","+38,858","1,018,718",+186,"80,506,860","+41,509","1,221,597","1,415","247,359","3,045","1,001,729,381","2,994,507","334,522,343",North America,4,328,0,116,0.6,"3,652"
3,India,"43,062,097","+2,011","522,223",,"42,523,311","+1,970","16,563",698,"30,658",372,"834,717,702","594,272","1,404,606,308",Asia,33,"2,690",2,1,,12
4,Brazil,"30,355,919","+6,456","662,777",+76,"29,411,813","+27,459","281,329","8,318","140,994","3,078","63,776,166","296,221","215,299,307",South America,7,325,3,30,0.4,"1,307"
5,France,"28,317,915","+13,984","145,257",+197,"26,083,461","+202,981","2,089,197","1,677","432,101","2,216","266,484,045","4,066,260","65,535,419",Europe,2,451,0,213,3,"31,879"
6,Germany,"24,227,680","+86,980","134,817",+171,"21,425,200","+182,200","2,667,663","1,980","287,502","1,600","122,332,384","1,451,676","84,269,739",Europe,3,625,1,"1,032",2,"31,656"
7,UK,"21,978,198","+10,582","173,693",+114,"20,955,075","+172,725","849,430",339,"320,697","2,534","514,985,782","7,514,457","68,532,664",Europe,3,395,0,154,2,"12,395"
8,Russia,"18,144,788","+7,651","375,061",+159,"17,483,483","+8,855","286,244","2,300","124,239","2,568","273,400,000","1,871,988","146,047,929",Europe,8,389,1,52,1,"1,960"
9,S. Korea,"16,929,564","+34,370","22,243",+110,N/A,N/A,N/A,668,"329,695",433,"15,804,065","307,776","51,349,193",Asia,3,"2,309",3,669,2,"310,381"


In [4]:
# export csv file
date = (dt.date.today() + dt.timedelta(days = -2)).strftime('%Y%m%d')
if (os.path.exists('output') == False):
    os.mkdir('output')
df.to_csv('output/Coronavirus_' + date + '.csv', index = False)

# Preprocessing

In [5]:
# check data type of each column
def open_object_dtype(s):
    return set(s.apply(lambda x: type(x)))

for col in df.columns:
    print(col, '\t', open_object_dtype(df[col]))

Country,Other 	 {<class 'str'>}
TotalCases 	 {<class 'str'>}
NewCases 	 {<class 'str'>}
TotalDeaths 	 {<class 'str'>}
NewDeaths 	 {<class 'str'>}
TotalRecovered 	 {<class 'str'>}
NewRecovered 	 {<class 'str'>}
ActiveCases 	 {<class 'str'>}
Serious,Critical 	 {<class 'str'>}
Tot Cases/1M pop 	 {<class 'str'>}
Deaths/1M pop 	 {<class 'str'>}
TotalTests 	 {<class 'str'>}
Tests/1M pop 	 {<class 'str'>}
Population 	 {<class 'str'>}
Continent 	 {<class 'str'>}
1 Caseevery X ppl 	 {<class 'str'>}
1 Deathevery X ppl 	 {<class 'str'>}
1 Testevery X ppl 	 {<class 'str'>}
New Cases/1M pop 	 {<class 'str'>}
New Deaths/1M pop 	 {<class 'str'>}
Active Cases/1M pop 	 {<class 'str'>}


In [6]:
# preprocessing value to number
df[['NewCases', 'NewDeaths', 'NewRecovered']] = \
    df[['NewCases', 'NewDeaths', 'NewRecovered']].applymap(lambda x: x.replace('+', ''))
df[df.columns.drop(['Country,Other', 'Continent'])] = \
    df[df.columns.drop(['Country,Other', 'Continent'])].applymap(lambda x: x.replace(',', '').replace(' ','')).replace('','0')

In [7]:
# preprocessing N/A to mean/median
df[['Country,Other', 'Continent']] = \
    df[['Country,Other', 'Continent']].replace('','N/A')
df = df.replace('N/A', np.nan)

In [8]:
# preprocessing change data type
def to_int(s):
    return [int(i) if not(i is np.nan) else np.nan for i in s]
def to_float(s):
    return [float(i) if not(i is np.nan) else np.nan for i in s]

for col in df.columns.drop(['Country,Other', 'Continent']):
    if col in ['New Cases/1M pop', 'New Deaths/1M pop', 'Active Cases/1M pop']:
        df[col] = to_float(df[col])
    else:
        df[col] = to_int(df[col])

In [9]:
# fill missing values by mean
for col in df.columns.drop(['Country,Other', 'Continent']):
    df[col].fillna(int(df[col].mean()), inplace = True)
    
df['TotalRecovered'] = to_int(df['TotalRecovered'])
df['NewRecovered'] = to_int(df['NewRecovered'])
df['ActiveCases'] = to_int(df['ActiveCases'])

In [10]:
df.dtypes

Country,Other           object
TotalCases               int64
NewCases                 int64
TotalDeaths              int64
NewDeaths                int64
TotalRecovered           int64
NewRecovered             int64
ActiveCases              int64
Serious,Critical         int64
Tot Cases/1M pop         int64
Deaths/1M pop            int64
TotalTests               int64
Tests/1M pop             int64
Population               int64
Continent               object
1 Caseevery X ppl        int64
1 Deathevery X ppl       int64
1 Testevery X ppl        int64
New Cases/1M pop       float64
New Deaths/1M pop      float64
Active Cases/1M pop    float64
dtype: object

In [11]:
df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
#,,,,,,,,,,,,,,,,,,,,,
1,China,203334,2680,4776,51,169380,2982,29178,274,141,3,160000000,111163,1439323776,Asia,7079,301366,9,2.00,0.04,20.00
2,USA,82747175,38858,1018718,186,80506860,41509,1221597,1415,247359,3045,1001729381,2994507,334522343,North America,4,328,0,116.00,0.60,3652.00
3,India,43062097,2011,522223,0,42523311,1970,16563,698,30658,372,834717702,594272,1404606308,Asia,33,2690,2,1.00,0.00,12.00
4,Brazil,30355919,6456,662777,76,29411813,27459,281329,8318,140994,3078,63776166,296221,215299307,South America,7,325,3,30.00,0.40,1307.00
5,France,28317915,13984,145257,197,26083461,202981,2089197,1677,432101,2216,266484045,4066260,65535419,Europe,2,451,0,213.00,3.00,31879.00
6,Germany,24227680,86980,134817,171,21425200,182200,2667663,1980,287502,1600,122332384,1451676,84269739,Europe,3,625,1,1032.00,2.00,31656.00
7,UK,21978198,10582,173693,114,20955075,172725,849430,339,320697,2534,514985782,7514457,68532664,Europe,3,395,0,154.00,2.00,12395.00
8,Russia,18144788,7651,375061,159,17483483,8855,286244,2300,124239,2568,273400000,1871988,146047929,Europe,8,389,1,52.00,1.00,1960.00
9,S. Korea,16929564,34370,22243,110,2121401,4925,86484,668,329695,433,15804065,307776,51349193,Asia,3,2309,3,669.00,2.00,310381.00


In [12]:
# export csv file
df.to_csv('output/data.csv', index = False)